# Pycuda-euler 

GPU assembler using PySpark for distributed computing

In [1]:
from pyspark import SparkContext
#sc.addPyFile("eulercuda.zip")

In [2]:
import subprocess

In [ ]:
import sys
sys.path.append('/home/ubuntu/pycuda-euler/src')

In [3]:
import eulercuda as ec


In [4]:
import pycuda.driver
import pycuda.autoinit

In [ ]:
# sc = SparkContext(master='local[*]')

In [ ]:
sc.defaultParallelism

In [ ]:
sc.defaultMinPartitions

In [5]:
complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}

Load data and check how many ways Spark/HDFS slices it up...

In [6]:
raw_data = sc.textFile('hdfs://172.31.26.32:8020/genome/Ba10k.sim1.fq',10)
raw_data.getNumPartitions()
# raw_data.count()

10

Data will either be in "fasta" or "fastq" format, meaning it will have more than just the DNA letters in it. We'll have to clean that up. 

**fastq parsing**

In [7]:
data = raw_data.filter(lambda x: len(x) > 0 and x[0] in ['A','C','G','T'])
dataLength = len(data.take(1)[0])
dataCount = data.count() // data.getNumPartitions()


The k value determines how long the dna substrings are. K value is always an odd number and is usually slightly more than 1/2 the size of the read length (read length is number of DNA characters on one line in the data file).

It should be noted that although k value is the often-seen metric in the literature, Mahmood's GPU-Euler called out an l ("ell") value, because l is one more than k and it is that lth character that forms the edge between two kmer nodes.

In [8]:
k = 17
lmerLength = 18

In [9]:
def hail_mary(path, data):
    import eulercuda.eulercuda as ec
    subprocess.call(["hdfs", "dfs", "-rm", "-r", "-f", path])
    hail_mary = data.mapPartitions(lambda x: ec.assemble2(k, buffer=x, readLength = dataLength,readCount=dataCount)).saveAsTextFile('hdfs://172.31.26.32/genome/gpu_Ba10k_output')

In [10]:
hail_mary('/genome/gpu_Ba10k_output',data)

## "Medium-sized" 2.2GB file:

In [ ]:
import os

In [ ]:
raw_data = sc.textFile('hdfs://172.31.26.32:8020/genome/sra_data.fastq', 100)

In [ ]:
data = raw_data.filter(lambda x: len(x) > 0 and x[0] in ['A','C','G','T'])
dataLength = len(data.take(1)[0])
dataCount = data.count() // data.getNumPartitions()

In [ ]:
k = 21
lmerLength = 22

In [ ]:
hail_mary = data.mapPartitions(lambda x: ec.assemble2(k, buffer=x, readLength = dataLength,readCount=dataCount)).saveAsTextFile('hdfs://172.31.26.32:8020/genome/gpu_sra_output')

## Note: Cells below are individual functions and notes. Use "hail_mary" method above.

**fasta parsing**

In [ ]:
# placeholder cell

I would like to think I could simply replace eulercuda.py's assemble2() by replicating function calls here, but it probably won't be that easy. 

In [ ]:
data.take(2)

In [ ]:
readLength = len(data.take(1)[0])
readLength

In [ ]:
readCount = data.count() * readLength
readCount

The original C used a buffer that was one *really* long string. This could be... problematic.

In [ ]:
buffer = data.mapPartitions(lambda x: ''.join(x))

In [ ]:
buffer

In [ ]:
baseCount = buffer.count()

In [ ]:
baseCount

In [ ]:
buffer.getNumPartitions()

In [ ]:
glom = buffer.repartition(1)
glom.getNumPartitions()

In [ ]:
    evList = []
    eeList = []
    edgeCount = 0
    levEdgeList = []
    entEdgeList = []
    edgeCount = 0
    vertexCount = 0

#### Note: 
**Important distinction:** 
if we have a one-argument function:

    def myFunc(x):

we can call it like this:

    myNewRDD = rdd.map(myFunc)

and rdd will be passed to myFunc as parameter x. **But** if we have multiple parameters, we handle it with lambda:

    def myOtherFunc(x, y, z):
    
    newRDD = rdd.map(lambda j: myOtherFunc(j, y, z))


In [ ]:
# use lambda form above. Each buffer partition will be passed to constructDebruijnGraph
partitionReadCount = readCount // buffer.getNumPartitions()
rdd_graph = glom.mapPartitions(lambda x: ec.constructDebruijnGraph(x, partitionReadCount, readLength, lmerLength, evList, eeList, levEdgeList, entEdgeList, readCount))
                                
et_graph= rdd_graph.mapPartitions(lambda x: ec.findEulerTour(x[1], x[0], x[2], x[3], x[5], x[4], lmerLength, "read1_spark.txt"))


In [ ]:
# rdd_Graph.getNumPartitions()
# evList.tolist()
#%time 
# rdd_Graph.collect()
# et_graph.collect()

operation above gets to setup_edges_device before erroring out. 

Based on console messages we're not calculating edges properly

Maybe try calling functions one by one?

In [ ]:
ø

In [ ]:
#findEulerTour(evList, eeList, levEdgeList, entEdgeList, edgeCount, vertexCount, lmerLength, outfile)
rdd_tour = rdd_Graph.mapPartitions(lambda x: ec.findEulerTour(x[1], x[0], x[2], x[3], x[5], x[4], lmerLength, "ba_spark.txt"))

In [ ]:
#rdd_tour.collect()

_constructDebruijnGraph_ functions:

    vals = readLmersKmersCuda(readBuffer, readLength, readCount, lmerLength, h_lmerKeys, h_lmerValues,
                              lmerCount, h_kmerKeys, h_kmerValues, kmerCount, numReads)
    results = gh.create_hash_table_device(h_kmerKeys, h_kmerValues, kmerCount, d_TK, d_TV, tableLength,
                                          d_bucketSize, bucketCount)

    tableLength = results[0]
    d_bucketSize = results[1]
    bucketCount = results[2]
    d_TK = results[3]       # kmer keys in sorted buckets
    d_TV = results[4]       # kmer values in sorted buckets

    d_ev, d_ee, d_levEdge, d_entEdge, kmerCount, edgeCount = \
        db.construct_debruijn_graph_device(h_lmerKeys, h_lmerValues, lmerCount, h_kmerKeys, kmerCount, lmerLength,
        d_TK, d_TV, d_bucketSize, bucketCount, d_ev, d_levEdge, d_entEdge, d_ee, readLength, readCount)


In [ ]:
# vals =  buffer.mapPartitions(lambda x: readLmersKmersCuda(x, readLength, readCount, lmerLength, h_lmerKeys, h_lmerValues,
#                               lmerCount, h_kmerKeys, h_kmerValues, kmerCount, numReads))

hail_mary = data.mapPartitions(lambda x: ec.assemble2(k, x)).collect()